In [1]:
import pandas as pd
import networkx as nx
import requests
import time
import re

In [5]:
# Get data
r = requests.get("https://api.pushshift.io/reddit/search/submission/?subreddit=politics&sort_type=score&sort=desc&after=1546300800&before=1577836800&size=500")
print(r.status_code)
print(r.headers['content-type'])

# Store data
submissions = r.json()['data']
df_sub = pd.DataFrame(submissions)

df_sub.to_pickle('submissions.pkl')

200
application/json; charset=UTF-8


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [ ]:
i = 0
df = pd.DataFrame()

for submission in submissions:
    comments = []

    date = submission['created_utc']
    r = requests.get("https://api.pushshift.io/reddit/search/comment/?subreddit=politics&sort=asc&after=" + str(date) + "&before=1577836800&link_id=" + submission['id'] + "&size=500")

    # As long as the API returns more data, keep getting more comments after the last comment we retrieved (sorted by date)
    while r.status_code == 200 and len(r.json()['data']) > 0:
        comments = comments + r.json()['data']
        date = comments[-1]['created_utc']

        r = requests.get("https://api.pushshift.io/reddit/search/comment/?subreddit=politics&sort=asc&after=" + str(date) + "&before=1577836800&link_id=" + submission['id'] + "&size=500")
        print(str(r.status_code) + '\n' if r.status_code != 200 else '.', end = '')
        time.sleep(0.4)

    # After getting all comments for a post, save them to create a point we can return to if necessary
    df = pd.concat([df, pd.DataFrame(comments)])
    df.to_pickle('all_comments.pkl')
    
#     print('')
#     print(i)
    i += 1
    
# Make sure to remove any duplicates
df = df.loc[~df['id'].duplicated()]



In [2]:
# If we already saved our comments
all_comments = pd.read_pickle('all_comments.pkl')

In [3]:
# Remove any duplicates
all_comments = all_comments.loc[~all_comments['id'].duplicated()]

In [4]:
import re
# Select important columns
all_comments = all_comments.filter(['author', 'body', 'id', 'link_id', 'parent_id', 'score'])

# remove 't3_' or 't1_' in references to other nodes 
for col in ['link_id', 'parent_id']:
    all_comments[col] = all_comments[col].apply(lambda str: re.sub(r'^t\d_', '', str))

print(all_comments['parent_id'].apply(lambda x: 't3_' in x or 't1_' in x).sum())

0


In [5]:
all_comments.to_pickle('all_comments.pkl')